In [1]:
import numpy as np
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
import time
from tqdm import tqdm

In [4]:
DATA_DIR = Path('./dataset/')

In [5]:
def calculate_2dft( input_img ):
    input = np.float32( input_img )[:,:,0]
    dft = np.fft.fft2(input)
    dft_shift = np.fft.fftshift(dft)
    ft = np.real(dft_shift)
    ft[int(ft.shape[0] / 2),int(ft.shape[1] / 2)] = 0
    
    ft = ft.reshape((ft.shape[0]*ft.shape[1],))
    ft /= ft.shape[0]

    return torch.Tensor(ft)

# Datenset laden

In [6]:
train_dir = "./dataset/train/"
test_dir = "./dataset/test/"
train_classa_dir = "./dataset/train/moire/"
train_classb_dir = "./dataset/train/no_moire/"
test_classa_dir = "./dataset/test/moire/"
test_classb_dir = "./dataset/test/no_moire/"

In [7]:
# Create transform function
transforms_train = transforms.Compose([
    transforms.Lambda(calculate_2dft)
])
transforms_test = transforms.Compose([
    transforms.Lambda(calculate_2dft)
])

In [8]:
train_dataset = datasets.ImageFolder(train_dir, transforms_train)
test_dataset = datasets.ImageFolder(test_dir, transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=48, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=48, shuffle=False, num_workers=0)

# Modell trainieren

In [9]:
#device = torch.device('mps')
device = torch.device('cpu')

In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(280 * 280, 20000)  # 5*5 from image dimension
        self.fc2 = nn.Linear(20000, 1000)
        self.fc3 = nn.Linear(1000, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

In [11]:
model = Net().to(device)

In [12]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [13]:
torch.backends.mps.is_available()

True

In [14]:
0 / 0

ZeroDivisionError: division by zero

In [15]:
#### Train model
train_loss=[]
train_accuary=[]
test_loss=[]
test_accuary=[]

num_epochs = 10   #(set no of epochs)
start_time = time.time() #(for showing time)
# Start loop
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch + 1)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    with tqdm(total=len(train_dataloader)) as pbar:       
        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device) 
            # forward inputs and get output
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            # get loss value and update the network weights
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data).item()

            pbar.update(1)
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    # Append result
    train_loss.append(epoch_loss)
    train_accuary.append(epoch_acc)
    # Print progress
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time() -start_time))
    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data).item()
        epoch_loss = running_loss / len(test_dataset)
        epoch_acc = running_corrects / len(test_dataset) * 100.
        # Append result
        test_loss.append(epoch_loss)
        test_accuary.append(epoch_acc)
        # Print progress
        print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 1 running


100%|█████████████████████████████████████████| 66/66 [54:55<00:00, 49.92s/it]


[Train #1] Loss: 0.0145 Acc: 51.5171% Time: 3295.0460s
[Test #1] Loss: 0.0150 Acc: 53.0075% Time: 3329.3774s
Epoch 2 running


100%|█████████████████████████████████████████| 66/66 [54:04<00:00, 49.17s/it]


[Train #2] Loss: 0.0144 Acc: 54.9305% Time: 6574.3380s
[Test #2] Loss: 0.0150 Acc: 54.8872% Time: 6609.9064s
Epoch 3 running


...

100%|███████████████████████████████████████████| 66/66 [1:38:32<00:00, 89.58s/it]


[Train #4] Loss: 0.0144 Acc: 61.8521% Time: 15714.4526s
[Test #4] Loss: 0.0150 Acc: 59.2105% Time: 15953.7965s
Epoch 5 running


100%|██████████████████████████████████████████| 66/66 [6:46:37<00:00, 369.65s/it]


[Train #5] Loss: 0.0144 Acc: 65.0759% Time: 40350.8717s
[Test #5] Loss: 0.0150 Acc: 61.7481% Time: 40405.4320s
Epoch 6 running


  0%|                                                | 0/66 [00:29<?, ?it/s]

KeyboardInterrupt

